In [ ]:
import numpy as np
import pandas as pd
import torch
import torchvision
from torchvision import datasets,models,transforms
from torch import nn
import torch.nn.functional as F
from torch import optim
import sklearn

In [ ]:
train_transforms=transforms.Compose([
                                     transforms.RandomRotation(30),
                                     transforms.Resize((224,224)),
                                    transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                            std=[0.229, 0.224, 0.225])])
test_transforms=transforms.Compose([
                                    transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                            std=[0.229, 0.224, 0.225])])

In [ ]:
train_data=datasets.ImageFolder('/content/drive/My Drive/DataSet/train',transform=train_transforms)
val_data=datasets.ImageFolder('/content/drive/My Drive/DataSet/validation',transform=test_transforms)
test_data=datasets.ImageFolder('/content/drive/My Drive/DataSet/test',transform=test_transforms)

In [ ]:
train_load=torch.utils.data.DataLoader(train_data,batch_size=100,shuffle=False)
val_load=torch.utils.data.DataLoader(val_data,batch_size=100,shuffle=False)
test_load=torch.utils.data.DataLoader(test_data,batch_size=100,shuffle=False)

In [ ]:
class Mish(nn.Module):
  def __init__(self):
    super().__init__()
  def forward(self,x):
    return x*(torch.tanh(F.softplus(x)))

In [ ]:
def save_checkpoint(state,is_best,filepath='/content/drive/My Drive/DataSet/resnet18.pth'):
  if is_best:
    print("==> Saving new best")
    torch.save(state,filepath)

In [ ]:
def load_checkpoint(checkpoint_fpath,model):
  checkpoint=torch.load(checkpoint_fpath)
  model.load_state_dict(checkpoint['state_dict'])
  return model, checkpoint['epoch'],checkpoint['last']

In [ ]:
device='cuda'
new_model=models.resnet18(pretrained=True)

num_labels=7
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    
    self.cnn_layer= torch.nn.Sequential(*(list(new_model.children())[:-1]))
    self.fc=torch.nn.Sequential(nn.Linear(512,7,bias=True),
                      
                                nn.LogSoftmax(dim=1))
    
  def forward(self,x):
    out=self.cnn_layer(x)
    out = out.view(out.size(0),-1)
    x1=out
    out=self.fc(out)
    return x1,out
model=Net()
model.to(device)

In [ ]:
criterion=nn.NLLLoss()
optimizer=optim.Adam(model.parameters(),lr=1e-4,weight_decay=0.0005)

In [ ]:
epochs=100
train_x=[]
val_x=[]
test_x=[]
# model,_,_=load_checkpoint('/content/drive/My Drive/DataSet/untitled2.pth',model)
trainloss_data=[]
accuracy_data=[]
valloss_data=[]
best_acc=0
best_val=1000
for epoch in range(epochs):
  train_loss=0
  val_loss=0
  accuracy=0

  model.train()
  counter=0
  for inputs,labels in train_load: 
    counter+=1
    inputs,labels=inputs.to(device),labels.to(device)
    optimizer.zero_grad()
    last,output=model.forward(inputs)
    train_x.append(last)
    loss=criterion(output,labels)
    
    loss.backward()
    optimizer.step()
    print(counter)
    train_loss+=loss.item()*inputs.size(0)
    
  model.eval()
  
  with torch.no_grad():
    counter=0
    val_loss=0
    for inputs,labels in val_load:
      counter+=1
      inputs,labels=inputs.to(device),labels.to(device)
      last,output=model.forward(inputs)
      valloss=criterion(output,labels)
      val_loss+=valloss*inputs.size(0)
      val_x.append(last)

      output=torch.exp(output)
      top_p,top_class=output.topk(1,dim=1)
      equals=top_class==labels.view(*top_class.shape)
      accuracy+=torch.mean(equals.type(torch.FloatTensor))
      print(counter)
  
  accuracy=accuracy/len(val_load)
  train_loss=train_loss/len(train_load.dataset)
  val_loss=val_loss/len(val_load.dataset)
  is_best = bool(best_acc<accuracy)
  print("Epoch : ",epoch+1)
  print(is_best)
  if is_best==True:
    best_acc=accuracy
    save_checkpoint({
          'epoch': epoch + 1,
          'state_dict': model.state_dict(),
        'best_accuracy': best_acc,
          'last':last
    }, is_best)
  


  trainloss_data.append(float('{:.3f}'.format(train_loss)))
  valloss_data.append(float('{:.3f}'.format(val_loss)))
  accuracy_data.append(float('{:.4f}'.format(accuracy)))
  print("Accuracy : {:.6f}".format(accuracy))
  print('Training Loss: {:.6f} \tValidation Loss: {:.6f}'.format(train_loss, val_loss))

In [ ]:
test_x=[]
model.eval()
model,_,_=load_checkpoint('/content/drive/My Drive/DataSet/resnet18.pth',model)
  
with torch.no_grad():
  counter=0
  test_loss=0
  accuracy=0
  for inputs,labels in test_load:
    counter+=1
    inputs,labels=inputs.to(device),labels.to(device)
    last,output=model.forward(inputs)
    testloss=criterion(output,labels)
    test_loss+=testloss*inputs.size(0)
    test_x.append(last)

    output=torch.exp(output)
    top_p,top_class=output.topk(1,dim=1)
    equals=top_class==labels.view(*top_class.shape)
    accuracy+=torch.mean(equals.type(torch.FloatTensor))
    print(counter)

accuracy=accuracy/len(test_load)

test_loss=test_loss/len(test_load.dataset)

print("Accuracy : {:.6f}".format(accuracy))
print("Test Loss: {:.6f}".format(test_loss))

In [ ]:
model,_,_=load_checkpoint('/content/drive/My Drive/DataSet/resnet18.pth',model)

train_2_transforms=transforms.Compose([
                                    transforms.Resize((224,224)),
                                     transforms.ToTensor()
                                     ])
test_2_transforms=transforms.Compose([
                                    transforms.Resize((224,224)),
                                    transforms.ToTensor()])

train_2_data=datasets.ImageFolder('/content/drive/My Drive/DataSet/train',transform=train_2_transforms)
val_2_data=datasets.ImageFolder('/content/drive/My Drive/DataSet/validation',transform=test_2_transforms)
test_2_data=datasets.ImageFolder('/content/drive/My Drive/DataSet/test',transform=test_2_transforms)

train_load_2=torch.utils.data.DataLoader(train_2_data,batch_size=1,shuffle=False,num_workers=16)
val_load_2=torch.utils.data.DataLoader(val_2_data,batch_size=1,shuffle=False,num_workers=16)
test_load_2=torch.utils.data.DataLoader(test_2_data,batch_size=1,shuffle=False,num_workers=16)


In [ ]:
model=model.to('cuda')
model.eval()
model,_,_=load_checkpoint('/content/drive/My Drive/DataSet/resnet18.pth',model)
with torch.no_grad():
  counter=0
  test_loss=0
  accuracy=0
  for inputs,labels in train_load_2:
    counter+=1
    inputs,labels=inputs.to(device),labels.to(device)
    _,output=model.forward(inputs)
    testloss=criterion(output,labels)
    test_loss+=testloss*inputs.size(0)

    output=torch.exp(output)
    top_p,top_class=output.topk(1,dim=1)
    equals=top_class==labels.view(*top_class.shape)
    accuracy+=torch.mean(equals.type(torch.FloatTensor))
    print(counter)

accuracy=accuracy/len(train_load_2)

test_loss=test_loss/len(train_load_2.dataset)

print("Accuracy : {:.6f}".format(accuracy))
print("Train Loss: {:.6f}".format(test_loss))

In [ ]:
model.eval()
model,_,_=load_checkpoint('/content/drive/My Drive/DataSet/resnet18.pth',model)
with torch.no_grad():
  counter=0
  test_loss=0
  accuracy=0
  for inputs,labels in val_load_2:
    counter+=1
    inputs,labels=inputs.to(device),labels.to(device)
    _,output=model.forward(inputs)
    testloss=criterion(output,labels)
    test_loss+=testloss*inputs.size(0)

    output=torch.exp(output)
    top_p,top_class=output.topk(1,dim=1)
    equals=top_class==labels.view(*top_class.shape)
    accuracy+=torch.mean(equals.type(torch.FloatTensor))
    print(counter)

accuracy=accuracy/len(val_load_2)

test_loss=test_loss/len(val_load_2.dataset)

print("Accuracy : {:.6f}".format(accuracy))
print("Val Loss: {:.6f}".format(test_loss))

In [ ]:
model.eval()
model,_,_=load_checkpoint('/content/drive/My Drive/DataSet/resnet18.pth',model)
with torch.no_grad():
  counter=0
  test_loss=0
  accuracy=0
  for inputs,labels in test_load_2:
    counter+=1
    inputs,labels=inputs.to(device),labels.to(device)
    _,output=model.forward(inputs)
    testloss=criterion(output,labels)
    test_loss+=testloss*inputs.size(0)

    output=torch.exp(output)
    top_p,top_class=output.topk(1,dim=1)
    equals=top_class==labels.view(*top_class.shape)
    accuracy+=torch.mean(equals.type(torch.FloatTensor))
    print(counter)

accuracy=accuracy/len(test_load_2)

test_loss=test_loss/len(test_load_2.dataset)

print("Accuracy : {:.6f}".format(accuracy))
print("Test Loss: {:.6f}".format(test_loss))

In [ ]:
model.eval()
model,_,_=load_checkpoint('/content/drive/My Drive/DataSet/resnet18.pth',model)
data1=[]
data2=[]
data3=[]
label=[]
with torch.no_grad():
  for inputs,labels in train_load_2:
    inputs,labels=inputs.to(device),labels.to(device)
    last,output=model.forward(inputs)
    data1.append(last.cpu().numpy().tolist()[0])
    label.append(labels.cpu().numpy().tolist()[0])

  for inputs,labels in val_load_2:
    inputs,labels=inputs.to(device),labels.to(device)
    last,output=model.forward(inputs)
    data2.append(last.cpu().numpy().tolist()[0])
    label.append(labels.cpu().numpy().tolist()[0])
  
  for inputs,labels in test_load_2:
    inputs,labels=inputs.to(device),labels.to(device)
    last,output=model.forward(inputs)
    data3.append(last.cpu().numpy().tolist()[0])
    label.append(labels.cpu().numpy().tolist()[0])

In [ ]:
data=data1+data2+data3

In [ ]:
df=pd.DataFrame(data)
df.to_csv('/content/drive/My Drive/DataSet/herlev_resnet18.csv')
df2=pd.DataFrame(label)
df2.to_csv('/content/drive/My Drive/DataSet/label_resnet18.csv')
 